# Data preparation

Code authored by: Khushbu Pandit

### imports

In [3]:
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset

### load data

In [6]:
df = pd.read_csv('data/idea-title_pairs-preferences.csv')

In [8]:
df.head()

,idea,title_a,title_b,title_b_preferred
0,text embedding models,Top 3 Text Embedding Models Every Data Scienti...,Text Embedding Models: From Theory to Practice,1
1,multimodal RAG,Master Multimodal RAG in Just 10 Minutes,Why Multimodal RAG Is the Next Big Thing in AI,0
2,model pruning,The Surprising Benefits of Model Pruning for B...,Reduce Model Size & Boost Performance with Pru...,1
3,"Fractals, self similarity, and crinkliness",The Beauty of Fractals: Exploring Infinite Cri...,The Math Behind Fractals: Crinkliness and Beyond,1
4,Fine-tuning LLMs for computer use,The Ultimate Guide to LLM Fine-Tuning,Boost Your LLM Skills: Fine-Tuning for Compute...,0


### create prompt

In [11]:
template = lambda idea : f"""Given the YouTube video idea write an engaging title.

**Video Idea**: {idea}

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!"""

In [13]:
def idea_to_prompt(idea):
    return [{"role": "user", "content": template(idea.lower())}]

In [15]:
df['prompt'] = df['idea'].apply(idea_to_prompt)

### create chosen and rejected responses

In [18]:
def title_to_completion(title):
    return [{"role": "assistant", "content": title}]

In [20]:
# create chosen and rejected columns
df['chosen'] = np.where(df['title_b_preferred'] == 1, df['title_b'].apply(title_to_completion), df['title_a'].apply(title_to_completion))
df['rejected'] = np.where(df['title_b_preferred'] == 1, df['title_a'].apply(title_to_completion), df['title_b'].apply(title_to_completion))

In [22]:
df.iloc[:,-3:].head()

,prompt,chosen,rejected
0,"[{'role': 'user', 'content': 'Given the YouTub...","[{'role': 'assistant', 'content': 'Text Embedd...","[{'role': 'assistant', 'content': 'Top 3 Text ..."
1,"[{'role': 'user', 'content': 'Given the YouTub...","[{'role': 'assistant', 'content': 'Master Mult...","[{'role': 'assistant', 'content': 'Why Multimo..."
2,"[{'role': 'user', 'content': 'Given the YouTub...","[{'role': 'assistant', 'content': 'Reduce Mode...","[{'role': 'assistant', 'content': 'The Surpris..."
3,"[{'role': 'user', 'content': 'Given the YouTub...","[{'role': 'assistant', 'content': 'The Math Be...","[{'role': 'assistant', 'content': 'The Beauty ..."
4,"[{'role': 'user', 'content': 'Given the YouTub...","[{'role': 'assistant', 'content': 'The Ultimat...","[{'role': 'assistant', 'content': 'Boost Your ..."


In [24]:
# write data to file
df.to_csv('data/preferences.csv')

### create train-valid split

In [27]:
# shuffle dataframe
df_shuffled = df.iloc[:,-3:].sample(frac=1, random_state=42).reset_index(drop=True)

# 90-10 split
train_size = int(0.9 * len(df_shuffled))

# slice accordingly
df_train = df_shuffled.iloc[:train_size]
df_valid = df_shuffled.iloc[train_size:]

In [29]:
# Convert the pandas DataFrames back to Hugging Face Datasets
train_ds = Dataset.from_pandas(df_train)
valid_ds = Dataset.from_pandas(df_valid)

# Combine into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_ds,
    'valid': valid_ds,
})

In [33]:
# push data to hub

dataset_dict.push_to_hub("khushbu3107/youtube-titles-dpo")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/khushbu3107/youtube-titles-dpo/commit/7acf2a7fd6bfd649e7e610e89de72fb9a0a5feef', commit_message='Upload dataset', commit_description='', oid='7acf2a7fd6bfd649e7e610e89de72fb9a0a5feef', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/khushbu3107/youtube-titles-dpo', endpoint='https://huggingface.co', repo_type='dataset', repo_id='khushbu3107/youtube-titles-dpo'), pr_revision=None, pr_num=None)

In [ ]:
774778777